In [1]:
#import geopandas as gpd
import dask_geopandas as dg
import rioxarray
from xrspatial.zonal import stats
import xarray as xr
from dask.distributed import Client, LocalCluster
from shapely.geometry import box
from rasterio.features import rasterize

C:\Users\thudso02\AppData\Roaming\Python\Python312\site-packages\cupy\_environment.py:216: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(


In [ ]:
# Spin up local dask cluster
cluster = LocalCluster(n_workers=16)
client = Client(cluster)
client.dashboard_link

'http://127.0.0.1:8787/status'

In [3]:
vpu_id = '0104'
layer = 'filldepth'

In [6]:
# Define paths to in zone data and landscape layer raster
inZoneData = f'../high_res_data/NHDPLUS_H_{vpu_id}_HU4_GDB.gdb'
LandscapeLayer = f'../high_res_data/NHDPLUS_H_{vpu_id}_HU4_RASTERS/HRNHDPlusRasters{vpu_id}/{layer}.tif'

In [7]:
# Load raster data
ll_array = rioxarray.open_rasterio(LandscapeLayer, chunks=True, lock=False).sel(band=1).drop_vars('band')
ll_array

<xarray.DataArray (y: 14596, x: 15347)> Size: 896MB
dask.array<getitem, shape=(14596, 15347), dtype=int32, chunksize=(5760, 5760), chunktype=numpy.ndarray>
Coordinates:
  * x            (x) float64 123kB 1.916e+06 1.916e+06 ... 2.07e+06 2.07e+06
  * y            (y) float64 117kB 2.746e+06 2.746e+06 ... 2.6e+06 2.6e+06
    spatial_ref  int32 4B 0
Attributes: (12/13)
    AREA_OR_POINT:           Area
    DataType:                Generic
    RepresentationType:      THEMATIC
    STATISTICS_MAXIMUM:      499960
    STATISTICS_MEAN:         161.60983393432
    STATISTICS_MINIMUM:      0
    ...                      ...
    STATISTICS_SKIPFACTORY:  1
    STATISTICS_STDDEV:       1761.4748328907
    _FillValue:              -2147483648
    scale_factor:            1.0
    add_offset:              0.0
    units:                   centimetre

In [8]:
izd_df = dg.read_file(inZoneData, layer='NHDPlusCatchment', npartitions=64)
izd_df

,nhdplusid,sourcefc,gridcode,areasqkm,vpuid,SHAPE_Length,SHAPE_Area,geometry
npartitions=64,,,,,,,,
0,float64,object,int32,float64,object,float64,float64,geometry
244,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...
15372,...,...,...,...,...,...,...,...
15559,...,...,...,...,...,...,...,...


In [9]:
# Convert vector gdf to raster data form
# vector data = izd_df
# raster = ll_array
izd_df = izd_df.to_crs(ll_array.rio.crs)
bounds = ll_array.rio.bounds()
bbox = box(*bounds)
vector_data_clipped = izd_df[izd_df.geometry.intersects(bbox)]
vector_data_clipped

,nhdplusid,sourcefc,gridcode,areasqkm,vpuid,SHAPE_Length,SHAPE_Area,geometry
npartitions=64,,,,,,,,
0,float64,object,int32,float64,object,float64,float64,geometry
244,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...
15372,...,...,...,...,...,...,...,...
15559,...,...,...,...,...,...,...,...


In [11]:
transform = ll_array.rio.transform()
width = ll_array.sizes['x']
height = ll_array.sizes['y']

shapes = [(geom, value) for geom, value in zip(vector_data_clipped.geometry, vector_data_clipped['gridcode'])]
len(shapes)

15560

In [12]:
# Rasterize izd_df shapes, into numpy array
zones = rasterize(
        shapes=shapes,
        out_shape=(height, width),
        transform=transform,
        fill=0,  # Fill value for areas outside the vector data
        dtype='int32'
    )
zones

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [13]:
zones_da = xr.DataArray(
        data=zones,
        dims=['y', 'x'],
        coords={
            'y': ll_array.y,
            'x': ll_array.x
        }
    ).chunk(ll_array.chunksizes)
zones_da

<xarray.DataArray (y: 14596, x: 15347)> Size: 896MB
dask.array<xarray-<this-array>, shape=(14596, 15347), dtype=int32, chunksize=(5760, 5760), chunktype=numpy.ndarray>
Coordinates:
  * y        (y) float64 117kB 2.746e+06 2.746e+06 2.746e+06 ... 2.6e+06 2.6e+06
  * x        (x) float64 123kB 1.916e+06 1.916e+06 ... 2.07e+06 2.07e+06

In [ ]:
stats_df = stats(
        zones=zones_da,
        values=ll_array,
        stats_funcs=["mean", "max", "mean", "sum", "count"],
        nodata_values=ll_array.rio.nodata
    )
stats_df

,zone,mean,max,min,sum,std,var,count
npartitions=1,,,,,,,,
,int32,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...


In [15]:
final_df = stats_df.compute()
final_df

C:\Users\thudso02\AppData\Roaming\Python\Python312\site-packages\distributed\client.py:3362: UserWarning: Sending large graph of size 854.54 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


,zone,mean,max,min,sum,std,var,count
0,0,NaN,NaN,NaN,0.0,NaN,NaN,0.0
1,1,NaN,NaN,NaN,0.0,NaN,NaN,0.0
2,2,NaN,NaN,NaN,0.0,NaN,NaN,0.0
3,3,26.642857,99.0,1.0,373.0,35.941634,1291.801020,14.0
4,4,NaN,NaN,NaN,0.0,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...
15556,15697,NaN,NaN,NaN,0.0,NaN,NaN,0.0
15557,15698,16.000000,53.0,2.0,192.0,14.888474,221.666667,12.0
15558,15699,NaN,NaN,NaN,0.0,NaN,NaN,0.0
15559,15700,12.576923,56.0,1.0,981.0,14.758326,217.808185,78.0


In [16]:
final_df.head()

,zone,mean,max,min,sum,std,var,count
0,0,NaN,NaN,NaN,0.0,NaN,NaN,0.0
1,1,NaN,NaN,NaN,0.0,NaN,NaN,0.0
2,2,NaN,NaN,NaN,0.0,NaN,NaN,0.0
3,3,26.642857,99.0,1.0,373.0,35.941634,1291.80102,14.0
4,4,NaN,NaN,NaN,0.0,NaN,NaN,0.0


In [17]:
final_df.to_csv(f'high_res_zonal_stats_dask_{layer}_{vpu_id}.csv')